# Trainingspipeline 3

- ohne Bigramme
- vector_size: 300
- window: 10
- min_count: 2
- seed: 1

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../../Korpus/Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [4]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
0,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0,iv . 11 . rima del arcade : tirsi leucasio ( g...
1,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0,"iv . 5 . Lorenzo magalotti , lettere odoroso (..."
2,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit..."
3,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...
4,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit..."


In [5]:
df.shape

(304129, 10)

In [6]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [7]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [8]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1985']
df10 = df_periods['1986-2000']
df11 = df_periods['2001-2021']

In [9]:
df9.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
74265,diacorisBiagi1.txt,DiaCoris,Enzo Biagi,L'alluvione del Polesine,1951.0,1951-1985,stampa,L ' angoscia di un padre Si parla di battaglie...,152.0,l ' angoscia di uno padre si parlare di battag...
74266,LISCastiglia1.txt,LIS,Roderigo di Castiglia,«Vittorini se n'è ghiuto/ E soli ci ha lasciat...,1951.0,1951-1985,stampa,"A dire il vero , nelle nostre file pochi se ne...",1695.0,"a dire il vero , nella nostro file pochi se ne..."
74267,LISBianchi1.txt,LIS,Ranuccio Bianchi Bandinelli,Il nostro lavoro nella scuola,1951.0,1951-1985,stampa,"Come in tutti i congressi , anche nel VII Cong...",2898.0,"come in tutto il congresso , anche nel vii con..."
74268,LLAlbertelli1.txt,Liber Liber,Pilo Albertelli,Rousseau,1951.0,1951-1985,prosa letteraria,﻿Pilo Albertelli\nRousseau\n\n Nacque il 28 g...,4894.0,﻿pilo albertelli \n rousseau \n\n nascere il...
74269,LISJotti1.txt,LIS,Nilde Jotti,La questione dei fumetti,1951.0,1951-1985,stampa,Il dibattito sulla stampa a fumetti per i raga...,2785.0,il dibattito sulla stampa a fumetto per il rag...


## Training von Word2Vec

In [10]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [11]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 10                        # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

### Zeitraum 1: 1700-1750

In [12]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [13]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 17.7 s


In [14]:
print(sentences1[3])

['lungo', 'inospita', 'campagna', 'aggiungere', 'invidia', 'lato', 'manco', 'dire', 'anch', 'essere', 'teco', 'labbro', 'bianco', 'veggo', 'veneno', 'cor', 'stagno']


In [15]:
len(sentences1)

113328

In [16]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 57s


In [17]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('riempiè', 0.7313293218612671),
 ('macello', 0.7270107269287109),
 ('trucidare', 0.7196223139762878),
 ('costernazione', 0.7002019286155701),
 ('vittorioso', 0.6966360807418823),
 ('sconfiggere', 0.6949829459190369),
 ('sarmati', 0.6916698813438416),
 ('saccheggiare', 0.6872434020042419),
 ('torma', 0.6872117519378662),
 ('sconfitta', 0.6819703578948975)]

In [18]:
# trainiertes Modell speichern

w2v1.save(os.path.join('trained_models/Word2Vec3', '3w2v1.model'))

### Zeitraum 2: 1751-1800

In [19]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [20]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 19.9 s


In [21]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 37s


In [22]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('atterrire', 0.8200985193252563),
 ('sgherro', 0.8127479553222656),
 ('cadde', 0.807887613773346),
 ('usurpatore', 0.806434690952301),
 ('tregua', 0.8027049899101257),
 ('ruina', 0.7996137142181396),
 ('letargo', 0.7956777215003967),
 ('estinguere', 0.7929683923721313),
 ('languente', 0.7923955917358398),
 ('respingere', 0.7916663289070129)]

In [23]:
w2v2.save(os.path.join('trained_models/Word2Vec3', '3w2v2.model'))

### Zeitraum 3: 1801-1825

In [24]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [25]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 16.8 s


In [26]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 48s


In [27]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('violentare', 0.7834270000457764),
 ('secreti', 0.7698948979377747),
 ('provocazione', 0.7665985822677612),
 ('abbattimento', 0.7648427486419678),
 ('incostanza', 0.7620207071304321),
 ('raddolcire', 0.7609381675720215),
 ('fraudi', 0.7588301301002502),
 ('orrore', 0.7580758333206177),
 ('reprimere', 0.7566009759902954),
 ('accecare', 0.7557209134101868)]

In [28]:
w2v3.save(os.path.join('trained_models/Word2Vec3', '3w2v3.model'))

### Zeitraum 4: 1826-1850

In [29]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [30]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 15.5 s


In [31]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 40s


In [32]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('scompiglio', 0.7537322640419006),
 ('gelosia', 0.7401814460754395),
 ('provocare', 0.7345261573791504),
 ('indegnazione', 0.7331358194351196),
 ('ribellione', 0.7250903844833374),
 ('opprimere', 0.7233288288116455),
 ('frenare', 0.7193193435668945),
 ('scoppiare', 0.7166538834571838),
 ('atterrire', 0.7142557501792908),
 ('furore', 0.7116437554359436)]

In [33]:
w2v4.save(os.path.join('trained_models/Word2Vec3', '3w2v4.model'))

### Zeitraum 5: 1851-1875

In [34]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [35]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 16.8 s


In [36]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 28s


In [37]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('opprimere', 0.7681309580802917),
 ('sopraffare', 0.767601728439331),
 ('fremito', 0.7647494077682495),
 ('straziante', 0.7640421986579895),
 ('straziare', 0.7619406580924988),
 ('subitaneo', 0.7618492841720581),
 ('sbigottire', 0.7605838179588318),
 ('terribil', 0.7600743174552917),
 ('fremere', 0.7577102184295654),
 ('comprimere', 0.7538847327232361)]

In [38]:
w2v5.save(os.path.join('trained_models/Word2Vec3', '3w2v5.model'))

### Zeitraum 6: 1876-1900

In [39]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [40]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 18.8 s


In [41]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 19s


In [42]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('incutere', 0.7789242267608643),
 ('sgomento', 0.75480055809021),
 ('angoscioso', 0.7448980808258057),
 ('parossismo', 0.7375725507736206),
 ('assalire', 0.7355402708053589),
 ('accesso', 0.7351625561714172),
 ('furore', 0.732271671295166),
 ('spaventevole', 0.7313284277915955),
 ('scatenare', 0.7304121255874634),
 ('crollo', 0.7280892729759216)]

In [43]:
w2v6.save(os.path.join('trained_models/Word2Vec3', '3w2v6.model'))

### Zeitraum 7: 1901-1925

In [44]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [45]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 17.9 s


In [46]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 24s


In [47]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('indicibile', 0.7170349955558777),
 ('subitaneo', 0.7104148864746094),
 ('irrefrenabile', 0.7006756663322449),
 ('spavento', 0.7006427049636841),
 ('orrore', 0.6975653171539307),
 ('indignazione', 0.6940968632698059),
 ('repugnanza', 0.6916726231575012),
 ('aspettazione', 0.6877546310424805),
 ('cocente', 0.6872572898864746),
 ('annichilire', 0.6867266297340393)]

In [48]:
w2v7.save(os.path.join('trained_models/Word2Vec3', '3w2v7.model'))

### Zeitraum 8: 1926-1950

In [49]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [50]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 17.3 s


In [51]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 34s


In [52]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('ribrezzo', 0.7778933048248291),
 ('spavento', 0.7688074707984924),
 ('raccapriccio', 0.7541230320930481),
 ('repentino', 0.7454419732093811),
 ('massimiliana', 0.7445576190948486),
 ('subitaneo', 0.7419242262840271),
 ('orrore', 0.7409265041351318),
 ('furore', 0.7397351264953613),
 ('contratte', 0.7364231944084167),
 ('sussulto', 0.7303903102874756)]

In [53]:
w2v8.save(os.path.join('trained_models/Word2Vec3', '3w2v8.model'))

### Zeitraum 9: 1951-1985

In [54]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [55]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 18.5 s


In [56]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 34s


In [57]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.6830013990402222),
 ('sanguinario', 0.6798822283744812),
 ('annientare', 0.6540356874465942),
 ('valanga', 0.6532155275344849),
 ('flagello', 0.6517100930213928),
 ('spirale', 0.6476256847381592),
 ('ferocia', 0.6467594504356384),
 ('terrificante', 0.6467075943946838),
 ('liberatorio', 0.6452987790107727),
 ('panico', 0.6407183408737183)]

In [58]:
w2v9.save(os.path.join('trained_models/Word2Vec3', '3w2v9.model'))

### Zeitraum 10: 1986-2000

In [59]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [60]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 35 s


In [61]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 33s


In [62]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('insanguinare', 0.7707791328430176),
 ('cieco', 0.7516153454780579),
 ('seminare', 0.7481915950775146),
 ('macchiare', 0.7330289483070374),
 ('sopravvissuto', 0.7285313010215759),
 ('affogare', 0.7262634634971619),
 ('inneggiare', 0.7224496603012085),
 ('lutto', 0.7220171689987183),
 ('sparo', 0.7204774618148804),
 ('spaventoso', 0.7179389595985413)]

In [63]:
w2v10.save(os.path.join('trained_models/Word2Vec3', '3w2v10.model'))

### Zeitraum 11: 2001-2021

In [64]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [65]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 12.7 s


In [66]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 23s


In [67]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('seminare', 0.8219045996665955),
 ('distruzione', 0.8120431303977966),
 ('orrore', 0.8044633865356445),
 ('barbaro', 0.8042702078819275),
 ('soffocare', 0.7971481084823608),
 ('devastante', 0.7959963083267212),
 ('prigioniero', 0.7831051349639893),
 ('crudele', 0.7823190689086914),
 ('odio', 0.7799502611160278),
 ('tormento', 0.7761210799217224)]

In [68]:
w2v11.save(os.path.join('trained_models/Word2Vec3', '3w2v11.model'))